In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader as Loader

import os

DATAFILE = '../data/study1.csv'
FORECASTSFILE = '../forecasts.yaml'

In [2]:
forecasts = load(open(FORECASTSFILE), Loader=Loader)
def get_true(key):
    path = os.path.join('..', forecasts[key]['filename'])
    return pd.read_csv(path).y.values[-1]

true_df = pd.DataFrame({
    'Forecast': forecasts.keys(),
    'True': [get_true(key) for key in forecasts.keys()]
})
true_df

,Forecast,True
0,COVID_cases,168.620000
1,Tsunamis,8.000000
2,Bitcoin,115.126000
3,SongTempo,120.525875
4,ClimateChange,47.000000
5,PoliceShootings,70.000000
6,Meteors,11.000000
7,TrumpTweets,156.000000
8,VaccineSearches,100.000000


In [3]:
df = pd.read_csv(DATAFILE)
df = df[(df.WorkerId != 'test') & (df.Status == 'Completed')].dropna(subset=['SecondEstimate50'])
df = df.merge(true_df, on='Forecast')
df.head()

,IPv4,WorkerId,Bootstrap,Context,SurveyCode,ID,EndTime,StartTime,Status,DemographicsTime,...,SecondEstimate95,ContextKnowledge,LookUp,AdditionalComments,Berlin2bTime,Berlin2b,Berlin3Time,Berlin3,next,True
0,173.22.23.45,A24LB89P1BPKKF,1.0,neither,O62F96,112,2021-03-18 17:52:38.413041,2021-03-18 17:33:08.729711,Completed,17.411276,...,50.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0
1,196.17.66.143,A1L7QPDYJJG98Y,1.0,both,zNcBYt,115,2021-03-18 18:04:04.849359,2021-03-18 17:50:45.277455,Completed,6.160700,...,95.0,2.0,1.0,NO COMMENTS,NaN,NaN,NaN,NaN,NaN,11.0
2,67.20.142.175,A26KZMPK2EKVMX,0.0,both,tCuBsE,118,2021-03-18 18:45:41.843004,2021-03-18 17:59:53.343141,Completed,18.374332,...,40.0,-1.0,0.0,Very interesting study.,NaN,NaN,NaN,NaN,NaN,11.0
3,67.164.74.58,ASC9DUCC64M3P,1.0,both,DPpr2Q,5,2021-03-18 14:02:31.107359,2021-03-18 13:37:43.694651,Completed,7.665935,...,250.0,-1.0,0.0,No additional comments. Thank you.,NaN,NaN,NaN,NaN,NaN,11.0
4,63.75.250.107,A6D4ZL7O2NH8M,0.0,neither,LgYlTV,13,2021-03-18 14:12:18.600962,2021-03-18 13:53:07.101269,Completed,19.097487,...,2.0,1.0,1.0,very nice,NaN,NaN,NaN,NaN,NaN,11.0


In [4]:
df = pd.wide_to_long(df, ['FirstEstimate', 'SecondEstimate'], i=['WorkerId', 'Forecast'], j='Percentile', suffix='\w+')\
    .reset_index()
df = df[df.Percentile != 'Time'] # columns named e.g. FirstEstimateTime also included in dataframe
df.Percentile = df.Percentile.astype(float) # previously object column because of 'Time'
df['AverageEstimate'] = (df.FirstEstimate + df.SecondEstimate) / 2
df['AverageEstimateLoss'] = (df['True'] - df.AverageEstimate) * (df.Percentile / 100 - (df['True'] < df.AverageEstimate))
df.head()

,WorkerId,Forecast,Percentile,CRT_WhalesTime,CRT_BatBall,CRT_StockTime,RaceWhite,CRT_StudentsCorrect,CRT_StockIntuitive,CRT_BatBallCorrect,...,CRT_Stock,IPv4,next,AdditionalComments,CRT_StudentsTime,CRT_StockCorrect,FirstEstimate,SecondEstimate,AverageEstimate,AverageEstimateLoss
0,A24LB89P1BPKKF,Meteors,5.0,16.479828,10.0,35.925735,1.0,1.0,0.0,0.0,...,loss,173.22.23.45,NaN,NaN,16.736469,1.0,4.0,4.0,4.0,0.350
1,A24LB89P1BPKKF,Meteors,50.0,16.479828,10.0,35.925735,1.0,1.0,0.0,0.0,...,loss,173.22.23.45,NaN,NaN,16.736469,1.0,19.0,25.0,22.0,5.500
2,A24LB89P1BPKKF,Meteors,95.0,16.479828,10.0,35.925735,1.0,1.0,0.0,0.0,...,loss,173.22.23.45,NaN,NaN,16.736469,1.0,45.0,50.0,47.5,1.825
4,A1L7QPDYJJG98Y,Meteors,5.0,2.443005,1.0,5.295950,1.0,0.0,0.0,0.0,...,loss,196.17.66.143,NaN,NO COMMENTS,6.631298,1.0,40.0,5.0,22.5,10.925
5,A1L7QPDYJJG98Y,Meteors,50.0,2.443005,1.0,5.295950,1.0,0.0,0.0,0.0,...,loss,196.17.66.143,NaN,NO COMMENTS,6.631298,1.0,65.0,50.0,57.5,23.250


In [5]:
# trim worst 5% of forecasts by forecast and percentile
quant_df = df.groupby(['Forecast', 'Percentile']).AverageEstimateLoss.quantile(.95)\
    .reset_index().rename(columns={'AverageEstimateLoss': 'AverageEstimateCutoff'})
df = df.merge(quant_df, on=['Forecast', 'Percentile'])
df = df[df.AverageEstimateLoss < df.AverageEstimateCutoff]
df.head()

,WorkerId,Forecast,Percentile,CRT_WhalesTime,CRT_BatBall,CRT_StockTime,RaceWhite,CRT_StudentsCorrect,CRT_StockIntuitive,CRT_BatBallCorrect,...,IPv4,next,AdditionalComments,CRT_StudentsTime,CRT_StockCorrect,FirstEstimate,SecondEstimate,AverageEstimate,AverageEstimateLoss,AverageEstimateCutoff
0,A24LB89P1BPKKF,Meteors,5.0,16.479828,10.0,35.925735,1.0,1.0,0.0,0.0,...,173.22.23.45,NaN,NaN,16.736469,1.0,4.0,4.0,4.0,0.350,36.575
1,A1L7QPDYJJG98Y,Meteors,5.0,2.443005,1.0,5.295950,1.0,0.0,0.0,0.0,...,196.17.66.143,NaN,NO COMMENTS,6.631298,1.0,40.0,5.0,22.5,10.925,36.575
2,A26KZMPK2EKVMX,Meteors,5.0,56.260748,5.0,119.828893,1.0,0.0,1.0,1.0,...,67.20.142.175,NaN,Very interesting study.,51.227580,0.0,40.0,30.0,35.0,22.800,36.575
3,ASC9DUCC64M3P,Meteors,5.0,14.452615,5.0,34.474927,1.0,1.0,0.0,1.0,...,67.164.74.58,NaN,No additional comments. Thank you.,25.481228,1.0,2.0,55.0,28.5,16.625,36.575
4,A6D4ZL7O2NH8M,Meteors,5.0,20.057617,1.0,29.949291,1.0,1.0,1.0,0.0,...,63.75.250.107,NaN,very nice,49.570824,0.0,22.0,5.0,13.5,2.375,36.575


In [6]:
# normalize dataframe
norm_df = df.groupby(['Forecast', 'Percentile']).AverageEstimateLoss.agg(['mean', 'std'])\
    .reset_index()
df = df.merge(norm_df, on=['Forecast', 'Percentile'])
df['AverageEstimateLoss'] = (df.AverageEstimateLoss - df['mean']) / df['std']
norm_df.head()

,Forecast,Percentile,mean,std
0,Bitcoin,5.0,12.878616,34.141399
1,Bitcoin,50.0,47.878789,11.257048
2,Bitcoin,95.0,82.050200,23.357001
3,COVID_cases,5.0,11.773941,21.829224
4,COVID_cases,50.0,62.479412,12.247350


In [7]:
# primary analysis
X, y = df[['Bootstrap', 'Context']].copy(), df.AverageEstimateLoss
X['Context'] = (X.Context == 'both').astype(int)
X['BootstrapXContext'] = X.Bootstrap * X.Context
X = sm.add_constant(X)
res = sm.OLS(y, X).fit().get_robustcov_results('cluster', groups=df.WorkerId)
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     AverageEstimateLoss   R-squared:                       0.011
Model:                             OLS   Adj. R-squared:                  0.005
Method:                  Least Squares   F-statistic:                     1.941
Date:                 Mon, 22 Mar 2021   Prob (F-statistic):              0.136
Time:                         12:09:30   Log-Likelihood:                -760.77
No. Observations:                  548   AIC:                             1530.
Df Residuals:                      544   BIC:                             1547.
Df Model:                            3                                         
Covariance Type:               cluster                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0213      0.080      0.267      0.791      -0.139       0.182
Bootstrap            -0.1888      0.123     -1.531      0.133      -0.437       0.059
Context               0.0988      0.109      0.907      0.369      -0.120       0.318
BootstrapXContext     0.0574      0.181      0.317      0.753      -0.307       0.421
==============================================================================
Omnibus:                       85.447   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              147.636
Skew:                           0.942   Prob(JB):                     8.74e-33
Kurtosis:                       4.708   Cond. No.                         6.88
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [8]:
# test for significance of bootstrapping, context
res.f_test(np.array([0, 1, 0, 1])), res.f_test(np.array([0, 0, 1, 1]))

(<class 'statsmodels.stats.contrast.ContrastResults'>
 <F test: F=array([[0.98770122]]), p=0.3255041541320689, df_denom=46, df_num=1>,
 <class 'statsmodels.stats.contrast.ContrastResults'>
 <F test: F=array([[1.16941679]]), p=0.28515788397789515, df_denom=46, df_num=1>)

In [9]:
df.groupby(['Bootstrap', 'Context'])[
    'Age', 
    'Male', 
    'LookUp', 
    'BerlinScore', 
    'CRT_BatBallIntuitive', 
    'AverageEstimateLoss'
].mean()

<ipython-input-9-0492641a0f31>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df.groupby(['Bootstrap', 'Context'])[


Age      Male    LookUp  BerlinScore  \
Bootstrap Context                                               
0.0       both     37.549020  0.673203  0.241830     2.143791   
          neither  45.201439  0.517986  0.309353     2.266187   
1.0       both     39.659420  0.579710  0.557971     1.652174   
          neither  34.906780  0.906780  0.627119     1.847458   

                   CRT_BatBallIntuitive  AverageEstimateLoss  
Bootstrap Context                                             
0.0       both                 0.431373             0.120077  
          neither              0.258993             0.021316  
1.0       both                 0.557971            -0.011378  
          neither              0.330508            -0.167496